<a href="https://colab.research.google.com/github/peter-callahan/flight-project/blob/main/Flight_Data_Feature_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import timeit
import string
import math
import datetime

import seaborn as sns
%matplotlib inline

from IPython.display import HTML, display

pd.set_option('display.max_columns', 500)

In [ ]:
flight_data_raw = pd.read_csv('./data/flight_data.csv')

In [ ]:
flight_data_raw.head(2)

,No,Latitude,Longitude,Altitude,Temperature,Speed,Course,Date,Time
0,1,31.858161,64.228278,1084.3,22.6,0.00,0.0,12/2/14,23:52:49
1,2,31.858087,64.228349,1084.3,22.6,10.58,140.8,12/2/14,23:52:50


##### Flight Data Track

![flight_track](https://callpete-public.s3.amazonaws.com/flight_project/images/flight_with_elevation_msl.png)

### Data Scrubbing

#### Rounding Lat/Long Values 

Our data consists of latitude/longitude values, altitude (meters), temperature (Celsius), speed (mph) and course (direction).  Each value is timestamped and observations are approximately 1 second apart each.

First, we need to create unique "location keys" to allow geographical data to be joined later in the analysis.  This permits the inclusion of surface elevation and weather observation data, both of which are dependent on location.

It is also necessary to round the latitude/longitude values so that observational data (to six decimal points) can be joined to elevation data (three decimal points).  We can only measure to our lowest level of accuracy.  Luckily, three decimal points is about 100M of accuracy and will suffice for this analysis. 

http://wiki.gis.com/wiki/index.php/Decimal_degrees

Thus, our first creation is the "location key" which uses truncated lat/long values to identify a location in our data set. 

In [ ]:
flight_data_raw['lat_key'] = flight_data_raw['Latitude'].apply(lambda x: '{0:.3f}'.format(float(x)))
flight_data_raw['long_key'] = flight_data_raw['Longitude'].apply(lambda x: '{0:.3f}'.format(float(x)))
flight_data_raw['location_key'] = flight_data_raw['lat_key'] + flight_data_raw['long_key']
flight_data_raw['location_key'] = flight_data_raw['location_key'].apply(lambda x: x.translate(str.maketrans("", "", string.punctuation)))

In [ ]:
flight_data_raw['Date'] = pd.to_datetime(flight_data_raw['Date'])
flight_data_raw['Time'] = pd.to_datetime(flight_data_raw['Time']).dt.time

flight_data_raw['Date_Time'] = flight_data_raw['Date'].astype(str) + ' ' + (flight_data_raw['Time']).astype(str)
flight_data_raw['Date_Time'] = pd.to_datetime( flight_data_raw['Date_Time'] )

In [ ]:
flight_data_raw.head()

,No,Latitude,Longitude,Altitude,Temperature,Speed,Course,Date,Time,lat_key,long_key,location_key,Date_Time
0,1,31.858161,64.228278,1084.3,22.6,0.00,0.0,2014-12-02,23:52:49,31.858,64.228,3185864228,2014-12-02 23:52:49
1,2,31.858087,64.228349,1084.3,22.6,10.58,140.8,2014-12-02,23:52:50,31.858,64.228,3185864228,2014-12-02 23:52:50
2,3,31.857789,64.228614,1084.3,22.6,20.82,142.9,2014-12-02,23:52:52,31.858,64.229,3185864229,2014-12-02 23:52:52
3,4,31.857726,64.228672,1084.3,22.6,8.81,142.1,2014-12-02,23:52:53,31.858,64.229,3185864229,2014-12-02 23:52:53
4,5,31.857692,64.228702,1084.3,22.6,4.75,143.1,2014-12-02,23:52:54,31.858,64.229,3185864229,2014-12-02 23:52:54


#### Removing NaNs

"Speed" is a measure of change in GPS coordinates over time. NaN values for "speed" accumulate while the GPS is initializing. This is bad data which can be safely dropped.

In [ ]:
flight_data_raw = flight_data_raw.dropna(subset=['Speed'])

#### Conversions 

Aviation (worldwide) measures altitude in feet and speed in knots.  The handheld GPS in this project recorded data in meters and mph so the necessary conversions must be made.

In [ ]:
def meter_to_feet(row):
    return int(round(row['Altitude'] * 3.28084))

def mph_to_groundspeed(row):
    return int(round(row['Speed'] * 0.868976))

flight_data_raw['Altitude_Feet_MSL'] = flight_data_raw.apply(meter_to_feet, axis=1)
flight_data_raw['Groundspeed'] = flight_data_raw.apply(mph_to_groundspeed, axis=1)

#### Elevation Data

This data came from NASA (https://asterweb.jpl.nasa.gov/gdem.asp) and includes latitude/longitude coordinates and surface elevation in meters.  The surface elevation is measured as MSL (Mean Sea Level), which describes the  elevation of the Earth's surface at a lat/long coordinate above the average elevation of the sea.  The Mean Sea Level of the sea is assumed to be 0 feet/meters. 

In [ ]:
elevation = pd.read_csv('./data/Elevation_Data_Final.csv')
elevation['lat'] = elevation['lat'].astype(float)
elevation['long'] = elevation['long'].astype(float)

# Meters to feet conversion for elevation data
elevation['elev_ft'] = elevation['elev'].apply(lambda x: int(round(x * 3.28084)))
elevation.head()

,lat,long,elev,elev_ft
0,33.0,63.000000,1273,4177
1,33.0,63.000833,1276,4186
2,33.0,63.001667,1274,4180
3,33.0,63.002500,1266,4154
4,33.0,63.003333,1287,4222


![image](https://callpete-public.s3.amazonaws.com/flight_project/images/elevation_overview.png)

Note the light brown areas areas of zero elevation - this is where no data for elevation data is available.  We are using open source data here and it can be incomplete.  We will deal with this gap later in the analysis.

#### Rounding Lat Long Values for Elevation Data + Prikey Creation

Similar to the work necessary for our flight data, lat/longs coordinates from the elevation data must be truncated so they can be merged with the flight data.  This will allow the combined dataset to know the altitude of the aircraft and the surface of the ground.

In [ ]:
# Truncating Lat/Longs
elevation['lat'] = elevation['lat'].apply(lambda x: '{0:.3f}'.format(float(x)))
elevation['long'] = elevation['long'].apply(lambda x: '{0:.3f}'.format(float(x)))

# Creating join key
elevation['prikey'] = elevation['lat'] + elevation['long']
elevation['prikey'] = elevation['prikey'].apply(lambda x: x.translate(str.maketrans("", "", string.punctuation)))

# Aligning types between elevation and flight data
elevation['prikey'] = elevation['prikey'].astype(int)
flight_data_raw['location_key'] = flight_data_raw['location_key'].astype(int)

Merge to select elevations - a better .apply method might be possible to smooth elevations in moutainous areas

In [ ]:
# Merge to select elevation 
flight_data = flight_data_raw.merge(elevation, how='left', left_on='location_key', right_on='prikey')

In [ ]:
flight_data.head()

,No,Latitude,Longitude,Altitude,Temperature,Speed,Course,Date,Time,lat_key,long_key,location_key,Date_Time,Altitude_Feet_MSL,Groundspeed,lat,long,elev,elev_ft,prikey
0,1,31.858161,64.228278,1084.3,22.6,0.00,0.0,2014-12-02,23:52:49,31.858,64.228,3185864228,2014-12-02 23:52:49,3557,0,31.858,64.228,886.0,2907.0,3.185864e+09
1,1,31.858161,64.228278,1084.3,22.6,0.00,0.0,2014-12-02,23:52:49,31.858,64.228,3185864228,2014-12-02 23:52:49,3557,0,31.858,64.228,882.0,2894.0,3.185864e+09
2,1,31.858161,64.228278,1084.3,22.6,0.00,0.0,2014-12-02,23:52:49,31.858,64.228,3185864228,2014-12-02 23:52:49,3557,0,31.858,64.228,885.0,2904.0,3.185864e+09
3,1,31.858161,64.228278,1084.3,22.6,0.00,0.0,2014-12-02,23:52:49,31.858,64.228,3185864228,2014-12-02 23:52:49,3557,0,31.858,64.228,884.0,2900.0,3.185864e+09
4,2,31.858087,64.228349,1084.3,22.6,10.58,140.8,2014-12-02,23:52:50,31.858,64.228,3185864228,2014-12-02 23:52:50,3557,9,31.858,64.228,886.0,2907.0,3.185864e+09


In [ ]:
df_size = flight_data.shape[0]
flight_data = flight_data.drop_duplicates(subset=['Date_Time'])
df_size_2 = flight_data.shape[0]

print("Duplicate Timestamps (dropped):{}".format(df_size - df_size_2))
print("Remaining Observations {}".format(df_size_2))
print("Missing Elevation Data {} ({:.2f}%)".format(flight_data['elev_ft'].isna().sum(), 
                                              (flight_data['elev_ft'].isna().sum()/df_size_2)*100))

Duplicate Timestamps (dropped):65203
Remaining Observations 135582
Missing Elevation Data 5679 (4.19%)


#### Feature Engineering - AGL and Takeoff Threshold

* **AGL** : "Above Ground Level" = MSL (Mean Sea Level) of GPS minus surface elevation.  This represents how far above the surface of the ground the aircraft is at any moment.  This is critical to determine whether a flight envelope is dangerous or safe, key to identifying "bad flying."

* **Takeoff Threshold** : Using a rolling average of groundspeed allows us to detect when the aircraft craft is moving fast enough to indicate a transition to flight.  Airspeeds on the ground are slow, thus a combination if elevation data and groundspeed allow us ignore spikey GPS data and identify more accurately if the aircraft is in flight or on the ground.

In [ ]:
flight_data['Takeoff_Threshold'] = flight_data['Groundspeed'].rolling(5).mean()
flight_data['Altitude_Feet_AGL'] = flight_data['Altitude_Feet_MSL'] - flight_data['elev_ft']

#### Identifying Individual Missions

A mission is an event where an aircraft takes off performs some action and comes back to land.  The aircraft could then shutdown (completing the mission) or refuel and relaunch, continuing the mission.  Here, individual missions are identified using the difference in time between observations.  A time difference of 1140 seconds is considered to be a different mission (~20 minutes).  This value was found by plotting the data and selecting a time that clearly marked the end of one mission and the start of another.

In situations like this the handhelf GPS was not turned off thus it could be difficult to tell if the aircraft was holding on the ground (lager) or refueling in preparation for another launch.  20 minutes is the minimum time necessary to perform a rapid refuel.  It would be uncommon to lager for longer than 20 minutes without refueling so the assumption is that every time a 20 minute delay is observed, it is part of a refuel to support a new mission.

In [ ]:
flight_data['delta'] = flight_data['Date_Time'].diff(periods=1)
flight_data['delta'] = flight_data['delta'].dt.total_seconds()

flight_data = flight_data.dropna(subset=['delta'])

In [ ]:
msn_number_start = 1

def msn_number(row):
    global msn_number_start

    if row['delta'] < 1140:
        return msn_number_start
    else:
        msn_number_start += 1
        return msn_number_start
    
flight_data['msn_nbr'] = flight_data.apply(msn_number, axis=1)

##### MSN Numbers 

![msn_numbers](https://callpete-public.s3.amazonaws.com/flight_project/images/msn_numbers.gif)

#### Correcting Systemic Error in "AGL"

AGL "Above Ground Level" cannot be negative for an aircraft.  By selecting the minimum altitude for a flight (which should be zero since the flight begins and ends on land) we can help correct some of the error from the basic GPS instrumentation.

Of note, missions take place on different days where the error for GPS instrumentation varies.  This could be due to any number of reasons (GPS coverage, Earth-Satellite distance, random atmospherics, signal attenuation, etc.)  Thus, correcting the error based on each day's error makes sense. 

In [ ]:
AGL_error = flight_data.groupby('msn_nbr')['Altitude_Feet_AGL'].min().reset_index()
AGL_error.head(10)

,msn_nbr,Altitude_Feet_AGL
0,1,568.0
1,2,-143.0
2,3,-130.0
3,4,-125.0
4,5,-183.0
5,6,-121.0
6,7,-65.0
7,8,-231.0
8,9,-82.0
9,10,-76.0


In [ ]:
flight_data = flight_data.merge(AGL_error, 
                                how='left', 
                                left_on='msn_nbr', 
                                right_on='msn_nbr')

In [ ]:
flight_data = flight_data.rename(columns={'Altitude_Feet_AGL_x': 'Altitude_Feet_AGL', 
                                          'Altitude_Feet_AGL_y': 'Altitude_Error'})

In [ ]:
flight_data['Altitude_Feet_AGL_Corr'] = flight_data['Altitude_Feet_AGL'] - flight_data['Altitude_Error']
flight_data[['msn_nbr', 'Altitude_Feet_AGL', 'Altitude_Feet_AGL_Corr']].head()

,msn_nbr,Altitude_Feet_AGL,Altitude_Feet_AGL_Corr
0,1,650.0,82.0
1,1,653.0,85.0
2,1,653.0,85.0
3,1,653.0,85.0
4,1,653.0,85.0


![image](https://callpete-public.s3.amazonaws.com/flight_project/images/flight_with_elevation_agl_raw_2.png)

#### Correcting Elevation Data Anomaly

Elevation data (source NASA satellite) jumps up sharply up for part of our data set.  The input data came in as a "coverage map" so elevation data became available for rectangular swathes of the map.  Thus, one of these elevation rectancles appears to be about 3000 feet higher than the nearby rectangles, an issues we can account for with the code below. 

This represents a kind of normalization, where an instantanious change of >2000 feet in elevation (an abrupt change in elevation) is assumed to be erronious.  Detection of such a condition is a trigger to reduce the value of elevation by the amount of the suspected failure (3000 feet).

##### Elevation Data Loss

![image](https://callpete-public.s3.amazonaws.com/flight_project/images/flight_with_elevation_agl_corr_2.png)

In [ ]:
# Elevation Data Correction
def elev_correction(row):
    global prev_elev
    
    # If the change between 2 data points (seperated by 1 second) is greater than 2000 feet, subtract 3000 feet
    if abs(row['Altitude_Feet_AGL_Corr'] - prev_elev) > 2000:
        prev_elev = row['Altitude_Feet_AGL_Corr'] - 3000
        return (prev_elev)
    else:
        prev_elev = row['Altitude_Feet_AGL_Corr']
        return row['Altitude_Feet_AGL_Corr']
    
def elev_correction_v2(row):
    if row['lat'] > 32.0 and row['long'] > 64.0:
        return row['Altitude Feet AGL'] - 3000
    else:
        return row['Altitude Feet AGL']
    
prev_elev = 500
flight_data['AGL_Final'] = flight_data.apply(elev_correction, axis=1)

![agl_correct_view](https://callpete-public.s3.amazonaws.com/flight_project/images/flight_with_elevation_agl_final.png)

#### Determining when Airborne

The GPS data is contained in a time series that starts with the aircraft on the ground, it then transitions to flight, and returns to land.  Identifying when an aircraft is airborne is necessary to identify what GPS points represent flight data compared with idle ground data.

* The aircraft can takeoff and land more than once during a single flight
* Taxing (moving the aircraft on the ground) involves flight for a helicopter that has [skids](https://blog.aopa.org/aopa/2011/03/30/wheels-or-skids/).  This is known as hover taxing and does not count as a "fligth event" in this analysis

In [ ]:
def airborne_detection(row):
    if (row['Takeoff_Threshold'] > 20):
        return 1
    elif (row['Takeoff_Threshold'] <= 20) & (row['AGL_Final'] > 100):
        return 1
    else:
        return 0

flight_data['airborne'] = flight_data.apply(airborne_detection, axis=1)

In [ ]:
#flight_data.to_csv('output_flight_data_hv.csv', index=False)

### Creating a "Weather Key"

Rounding time values (calibrated in seconds in the original data) allows weather data to be joined via time + nearest location to provide the best possible estimate of weather conditions at a given point in space of a flight. 

In [ ]:
def minute_time_round(row):
    #tm = datetime.datetime(2012,12,31,14,52,59,1234)
    tm = row['Date_Time']
    tm = tm - datetime.timedelta(minutes=tm.minute % 5,
                                 seconds=tm.second,
                                 microseconds=tm.microsecond)

    discard = datetime.timedelta(minutes=tm.minute % 5,
                                 seconds=tm.second,
                                 microseconds=tm.microsecond)
    tm -= discard
    
    if discard >= datetime.timedelta(minutes=5):
        tm += datetime.timedelta(minutes=10)

    return tm
    
def hour_time_round(row):
    return row['Date_Time'].replace(minute=0, second=0, microsecond=0)

In [ ]:
flight_data['Date_Time'] = pd.to_datetime(flight_data['Date_Time'])
flight_data['Date_Time_Key'] = flight_data.apply(hour_time_round, axis=1)
flight_data.head()

,No,Latitude,Longitude,Altitude,Temperature,Speed,Course,Date,Time,lat_key,long_key,location_key,Date_Time,Altitude_Feet_MSL,Groundspeed,lat,long,elev,elev_ft,prikey,Takeoff_Threshold,Altitude_Feet_AGL,delta,msn_nbr,Altitude_Error,Altitude_Feet_AGL_Corr,AGL_Final,airborne,Date_Time_Key
0,2,31.858087,64.228349,1084.3,22.6,10.58,140.8,2014-12-02,23:52:50,31.858,64.228,3185864228,2014-12-02 23:52:50,3557,9,31.858,64.228,886.0,2907.0,3.185864e+09,NaN,650.0,1.0,1,568.0,82.0,82.0,0,2014-12-02 23:00:00
1,3,31.857789,64.228614,1084.3,22.6,20.82,142.9,2014-12-02,23:52:52,31.858,64.229,3185864229,2014-12-02 23:52:52,3557,18,31.858,64.229,885.0,2904.0,3.185864e+09,NaN,653.0,2.0,1,568.0,85.0,85.0,0,2014-12-02 23:00:00
2,4,31.857726,64.228672,1084.3,22.6,8.81,142.1,2014-12-02,23:52:53,31.858,64.229,3185864229,2014-12-02 23:52:53,3557,8,31.858,64.229,885.0,2904.0,3.185864e+09,NaN,653.0,1.0,1,568.0,85.0,85.0,0,2014-12-02 23:00:00
3,5,31.857692,64.228702,1084.3,22.6,4.75,143.1,2014-12-02,23:52:54,31.858,64.229,3185864229,2014-12-02 23:52:54,3557,4,31.858,64.229,885.0,2904.0,3.185864e+09,7.8,653.0,1.0,1,568.0,85.0,85.0,0,2014-12-02 23:00:00
4,6,31.857658,64.228735,1084.3,22.6,4.91,140.8,2014-12-02,23:52:55,31.858,64.229,3185864229,2014-12-02 23:52:55,3557,4,31.858,64.229,885.0,2904.0,3.185864e+09,8.6,653.0,1.0,1,568.0,85.0,85.0,0,2014-12-02 23:00:00


#### Collecting Weather Data

This data is from historical weather archives from a base long forgotten to most of us.  Those who flew in this area will never forget it, luckily the data collected from this remote outpost was made publically available through [geographic.org](https://geographic.org/global_weather/afghanistan/fob_sabit_qadam_409764_99999.html).

In [ ]:
weather_data = pd.read_fwf('./data/sabbitqaddam_weather2013.txt')

In [ ]:
weather_data.head()

,USAF,WBAN,YR--MODAHRMN,DIR,SPD,GUS,CLG,SKC,L,M,H,VSB,MW,MW.1,MW.2,MW.3,AW,AW.1,AW.2,AW.3,W,TEMP,DEWP,SLP,ALT,STP,MAX,MIN,PCP01,PCP06,PCP24,PCPXX,SD
0,409764,99999,201312010055,340,5,***,722,SCT,*,*,*,6.2,**,**,**,**,**,**,**,**,*,46,41,1020.0,30.10,******,***,***,*****,*****,*****,*****,**
1,409764,99999,201312010155,340,5,***,722,SCT,*,*,*,6.2,**,**,**,**,**,**,**,**,*,45,41,1020.9,30.12,******,***,***,*****,*****,*****,*****,**
2,409764,99999,201312010255,340,3,***,200,***,*,*,*,6.2,**,**,**,**,**,**,**,**,*,45,41,1021.5,30.14,******,***,***,*****,*****,*****,*****,**
3,409764,99999,201312010355,990,2,***,200,***,*,*,*,6.2,**,**,**,**,**,**,**,**,*,50,41,1021.8,30.16,******,***,***,*****,*****,*****,*****,**
4,409764,99999,201312010455,990,5,***,200,***,*,*,*,6.2,**,**,**,**,**,**,**,**,*,49,42,1022.8,30.18,******,***,***,*****,*****,*****,*****,**


#### Variable Processing and Time Key Creation

In [ ]:
weather_data['YR--MODAHRMN'] = weather_data['YR--MODAHRMN'].astype(str)
weather_data['Date'] = weather_data['YR--MODAHRMN'].str[:8]
weather_data['Date'] = pd.to_datetime(weather_data['Date'])

weather_data['Time'] = weather_data['YR--MODAHRMN'].str[-4:-2] + ":" + weather_data['YR--MODAHRMN'].str[-2:] + ":00"
weather_data['Time'] = pd.to_datetime(weather_data['Time']).dt.time

weather_data['Date_Time'] = weather_data['Date'].astype(str) + " " + weather_data['Time'].astype(str)
weather_data['Date_Time'] = pd.to_datetime( weather_data['Date_Time'] )

weather_data['Date_Time_Key'] = weather_data.apply(hour_time_round, axis=1)

In [ ]:
weather_data.head()

,USAF,WBAN,YR--MODAHRMN,DIR,SPD,GUS,CLG,SKC,L,M,H,VSB,MW,MW.1,MW.2,MW.3,AW,AW.1,AW.2,AW.3,W,TEMP,DEWP,SLP,ALT,STP,MAX,MIN,PCP01,PCP06,PCP24,PCPXX,SD,Date,Time,Date_Time,Date_Time_Key
0,409764,99999,201312010055,340,5,***,722,SCT,*,*,*,6.2,**,**,**,**,**,**,**,**,*,46,41,1020.0,30.10,******,***,***,*****,*****,*****,*****,**,2013-12-01,00:55:00,2013-12-01 00:55:00,2013-12-01 00:00:00
1,409764,99999,201312010155,340,5,***,722,SCT,*,*,*,6.2,**,**,**,**,**,**,**,**,*,45,41,1020.9,30.12,******,***,***,*****,*****,*****,*****,**,2013-12-01,01:55:00,2013-12-01 01:55:00,2013-12-01 01:00:00
2,409764,99999,201312010255,340,3,***,200,***,*,*,*,6.2,**,**,**,**,**,**,**,**,*,45,41,1021.5,30.14,******,***,***,*****,*****,*****,*****,**,2013-12-01,02:55:00,2013-12-01 02:55:00,2013-12-01 02:00:00
3,409764,99999,201312010355,990,2,***,200,***,*,*,*,6.2,**,**,**,**,**,**,**,**,*,50,41,1021.8,30.16,******,***,***,*****,*****,*****,*****,**,2013-12-01,03:55:00,2013-12-01 03:55:00,2013-12-01 03:00:00
4,409764,99999,201312010455,990,5,***,200,***,*,*,*,6.2,**,**,**,**,**,**,**,**,*,49,42,1022.8,30.18,******,***,***,*****,*****,*****,*****,**,2013-12-01,04:55:00,2013-12-01 04:55:00,2013-12-01 04:00:00


In [ ]:
flight_data['Date_Time_Key'] = flight_data['Date_Time_Key'].astype(str)
weather_data['Date_Time_Key'] = weather_data['Date_Time_Key'].astype(str)

flight_data_wx = flight_data.merge(weather_data, 
                                   how='left', 
                                   left_on='Date_Time_Key', 
                                   right_on='Date_Time_Key')

In [ ]:
# Remove extra Wx datapoints
flight_data_test = flight_data_wx.dropna()

print('Post Wx Merge Loss')
print('NaN Values:', flight_data_test.shape[0] / flight_data_wx.shape[0])

flight_data_wx = flight_data_wx.dropna()

Post Wx Merge Loss
NaN Values: 0.8941352884675184


#### [REMOVE BEFORE FLIGHT] This is due to Wx dates dropping off in March 2014 - can you get more data from that website?

In [ ]:
flight_data_wx = flight_data_wx.rename(columns={'Time_x': 'Time', 'Date_Time_x': 'Date_Time', 'Date_y':'Date'})
flight_data_wx.head()

,No,Latitude,Longitude,Altitude,Temperature,Speed,Course,Date_x,Time,lat_key,long_key,location_key,Date_Time,Altitude_Feet_MSL,Groundspeed,lat,long,elev,elev_ft,prikey,Takeoff_Threshold,Altitude_Feet_AGL,delta,msn_nbr,Altitude_Error,Altitude_Feet_AGL_Corr,AGL_Final,airborne,Date_Time_Key,USAF,WBAN,YR--MODAHRMN,DIR,SPD,GUS,CLG,SKC,L,M,H,VSB,MW,MW.1,MW.2,MW.3,AW,AW.1,AW.2,AW.3,W,TEMP,DEWP,SLP,ALT,STP,MAX,MIN,PCP01,PCP06,PCP24,PCPXX,SD,Date,Time_y,Date_Time_y
412,414,31.857462,64.229172,1066.9,25.0,0.67,166.4,2014-02-13,00:00:00,31.857,64.229,3185764229,2014-02-13 00:00:00,3500,1,31.857,64.229,884.0,2900.0,3.185764e+09,0.2,600.0,-25315199.0,1,568.0,32.0,32.0,0,2014-02-13 00:00:00,409764.0,99999.0,201402130055,360,5.0,***,722,SCT,*,*,*,6.2,**,**,**,**,**,**,**,**,*,35.0,32.0,1017.0,29.97,******,***,***,*****,*****,*****,*****,**,2014-02-13,00:55:00,2014-02-13 00:55:00
413,415,31.857451,64.229171,1067.1,25.0,1.28,182.5,2014-02-13,00:00:01,31.857,64.229,3185764229,2014-02-13 00:00:01,3501,1,31.857,64.229,884.0,2900.0,3.185764e+09,0.4,601.0,1.0,1,568.0,33.0,33.0,0,2014-02-13 00:00:00,409764.0,99999.0,201402130055,360,5.0,***,722,SCT,*,*,*,6.2,**,**,**,**,**,**,**,**,*,35.0,32.0,1017.0,29.97,******,***,***,*****,*****,*****,*****,**,2014-02-13,00:55:00,2014-02-13 00:55:00
414,416,31.857443,64.229173,1067.2,25.0,0.85,168.7,2014-02-13,00:00:02,31.857,64.229,3185764229,2014-02-13 00:00:02,3501,1,31.857,64.229,884.0,2900.0,3.185764e+09,0.6,601.0,1.0,1,568.0,33.0,33.0,0,2014-02-13 00:00:00,409764.0,99999.0,201402130055,360,5.0,***,722,SCT,*,*,*,6.2,**,**,**,**,**,**,**,**,*,35.0,32.0,1017.0,29.97,******,***,***,*****,*****,*****,*****,**,2014-02-13,00:55:00,2014-02-13 00:55:00
415,417,31.857437,64.229171,1067.3,25.0,0.70,191.8,2014-02-13,00:00:03,31.857,64.229,3185764229,2014-02-13 00:00:03,3502,1,31.857,64.229,884.0,2900.0,3.185764e+09,0.8,602.0,1.0,1,568.0,34.0,34.0,0,2014-02-13 00:00:00,409764.0,99999.0,201402130055,360,5.0,***,722,SCT,*,*,*,6.2,**,**,**,**,**,**,**,**,*,35.0,32.0,1017.0,29.97,******,***,***,*****,*****,*****,*****,**,2014-02-13,00:55:00,2014-02-13 00:55:00
416,418,31.857426,64.229170,1067.4,25.0,1.25,185.1,2014-02-13,00:00:04,31.857,64.229,3185764229,2014-02-13 00:00:04,3502,1,31.857,64.229,884.0,2900.0,3.185764e+09,1.0,602.0,1.0,1,568.0,34.0,34.0,0,2014-02-13 00:00:00,409764.0,99999.0,201402130055,360,5.0,***,722,SCT,*,*,*,6.2,**,**,**,**,**,**,**,**,*,35.0,32.0,1017.0,29.97,******,***,***,*****,*****,*****,*****,**,2014-02-13,00:55:00,2014-02-13 00:55:00


#### Weather Data Cleaning

In [ ]:
def direction_checker(row):
    if row['DIR'] > 360:
        return 0
    else:
        return row['SPD']

flight_data_wx['DIR'] = flight_data_wx['DIR'].replace('***', 0)
flight_data_wx['DIR'] = flight_data_wx['DIR'].astype(int)
flight_data_wx['SPD'] = flight_data_wx.apply(direction_checker, axis=1)

##### Calculating Crosswind/Headwind Component

This allows us to determine the aircraft's path through the air, rather than the aircraft's path over the ground.  This matters because the aerodynamic properties that identify "bad flying" (discussed below) exist relative to the aircraft's speed through the air.  

To describe it simply, groundspeed matters when landing, indicated airspeed matters for flying. 

![cross_wind](https://callpete-public.s3.amazonaws.com/flight_project/images/crosswind_calculation.png)

In [ ]:
def headwind(row):
    if row['airborne'] == 0:
        return 0
    else:
        aircraft_course = int(row['Course'])
        wind_direction = int(row['DIR'])
        
        if aircraft_course > wind_direction:
            offset = abs(aircraft_course - wind_direction)
        
        else:
            offset = abs(wind_direction - aircraft_course)
            
        if offset == 0: return row['SPD']
            
        # Headwind component
        if offset > 0 & offset < 90:
            return np.cos(np.deg2rad(offset)) * row['SPD']
        
        # 100% crosswind component, no headwind or tailwind
        if offset == 90: return 0
        
        # Tailwind component (negative value, opposite of a headwind)
        if offset > 90:
            offset = offset - 90
            return -(np.tan(np.deg2rad(offset)) * row['SPD'])

flight_data_wx['Headwind_Tailwind'] = flight_data_wx.apply(headwind, axis=1)
flight_data_wx['Airspeed'] = flight_data_wx['Speed'] + flight_data_wx['Headwind_Tailwind']

### Measuring Height-Velocity Violations

What is the height-velocity diagram and why do we care about it?  Check out the link to the paper accompanying this analysis for more details.  The short version: height-velocity acts as a proxy for "bad flying" and is calculated in the following way: 

In [ ]:
def height_velocity_violation(row):
    error_var = 5
    
    if row['airborne'] == 0:
        return 0
    
    else:
        if row['AGL_Final'] <= 30:
            return 0
        
        elif (row['AGL_Final'] > 30) & (row['AGL_Final'] < 100):
            y1 = (3.5 * row['Airspeed']) - 145
            z = y1 - row['AGL_Final']
                  
            if z < error_var:
                return 1 
            else:
                return 0
                  
        elif (row['AGL_Final'] >= 100) & (row['AGL_Final'] < 200):
            if row['Airspeed'] < 70:
                return 1
            else:
                return 0
            
        elif (row['AGL_Final'] >= 200) & (row['AGL_Final'] < 1277):
            y1 = (-15.4 * row['Airspeed']) + 1277
            z = y1 - row['AGL_Final']
            
            # The z 'greater than' comparison switches here because the slope of the line reverses
            if z > error_var:
                return 1 
            else:
                return 0
            
        else:
            return 0

flight_data_wx['hv_violation'] = flight_data_wx.apply(height_velocity_violation, axis=1)

In [ ]:
display(HTML("<table><tr><td><img src='https://callpete-public.s3.amazonaws.com/flight_project/images/Height-Velocity+Diagram.png'></td><td><img src='https://callpete-public.s3.amazonaws.com/flight_project/images/hv_diagram_datapoints.png'></td></tr></table>"))

,


#### Determining "Threat Percentage" Value

This metric reflects the total number of observations of a height velocity violation / total number of data points.  This reflects the number of seconds spent observed in a poor flight condition as a percentage of the total flight.

In [ ]:
analysis = flight_data_wx.groupby('msn_nbr')['hv_violation'].agg(['sum', 'count'])
analysis = analysis.reset_index("msn_nbr")
analysis['Threat_Percentage'] = (analysis['sum'] / analysis['count']) * 100

In [ ]:
flight_data_fin = flight_data_wx.merge(analysis, 
                                      how='left', 
                                      left_on='msn_nbr', 
                                      right_on='msn_nbr')

flight_data_fin = flight_data_fin.drop(['sum', 'count'], axis=1)

In [ ]:
# Drop a useless msn number, probable GPS extra datapoint
flight_data_fin = flight_data_fin[ flight_data_fin['msn_nbr'] != 16]

columns = ['Time_y', 'GUS', 'YR--MODAHRMN','Date_Time_y', 'USAF', 'WBAN', 'prikey','L','M','H','MW','MW.1','MW.2','MW.3','AW','AW.1','AW.2','AW.3','W','STP','MAX','MIN','PCP01','PCP06','PCP24',
         'PCPXX', 'Date_Time_Key','SD', 'delta', 'Takeoff_Threshold', 'Date_x', 'Altitude', 'location_key', 'lat_key', 'long_key']

flight_data_fin = flight_data_fin.drop(columns, axis=1)

In [ ]:
flight_data_fin.head()

,No,Latitude,Longitude,Temperature,Speed,Course,Time,Date_Time,Altitude_Feet_MSL,Groundspeed,lat,long,elev,elev_ft,Altitude_Feet_AGL,msn_nbr,Altitude_Error,Altitude_Feet_AGL_Corr,AGL_Final,airborne,DIR,SPD,CLG,SKC,VSB,TEMP,DEWP,SLP,ALT,Date,Headwind_Tailwind,Airspeed,hv_violation,Threat_Percentage
0,414,31.857462,64.229172,25.0,0.67,166.4,00:00:00,2014-02-13 00:00:00,3500,1,31.857,64.229,884.0,2900.0,600.0,1,568.0,32.0,32.0,0,360,5.0,722,SCT,6.2,35.0,32.0,1017.0,29.97,2014-02-13,0.0,0.67,0,19.094978
1,415,31.857451,64.229171,25.0,1.28,182.5,00:00:01,2014-02-13 00:00:01,3501,1,31.857,64.229,884.0,2900.0,601.0,1,568.0,33.0,33.0,0,360,5.0,722,SCT,6.2,35.0,32.0,1017.0,29.97,2014-02-13,0.0,1.28,0,19.094978
2,416,31.857443,64.229173,25.0,0.85,168.7,00:00:02,2014-02-13 00:00:02,3501,1,31.857,64.229,884.0,2900.0,601.0,1,568.0,33.0,33.0,0,360,5.0,722,SCT,6.2,35.0,32.0,1017.0,29.97,2014-02-13,0.0,0.85,0,19.094978
3,417,31.857437,64.229171,25.0,0.70,191.8,00:00:03,2014-02-13 00:00:03,3502,1,31.857,64.229,884.0,2900.0,602.0,1,568.0,34.0,34.0,0,360,5.0,722,SCT,6.2,35.0,32.0,1017.0,29.97,2014-02-13,0.0,0.70,0,19.094978
4,418,31.857426,64.229170,25.0,1.25,185.1,00:00:04,2014-02-13 00:00:04,3502,1,31.857,64.229,884.0,2900.0,602.0,1,568.0,34.0,34.0,0,360,5.0,722,SCT,6.2,35.0,32.0,1017.0,29.97,2014-02-13,0.0,1.25,0,19.094978


In [ ]:
flight_data_fin.to_csv('output_flight_data_hv_wx.csv', index=False)